# Reading the raw data

In [0]:
df = spark.read.format('parquet').load("abfss://bronze@storageadlsgen3.dfs.core.windows.net/rawdata")

In [0]:
display(df.head(5))

# Transformation

In [0]:
from pyspark.sql.functions import split, col, sum

In [0]:
df = df.withColumn("model_category", split(col("Model_ID"),'-')[0])

In [0]:
display(df.head(5))

In [0]:
df.withColumn("Units_Sold", col("Units_Sold").cast("long")).printSchema()

In [0]:
df = df.withColumn("price", col("Revenue")/col("Units_Sold"))
display(df)

In [0]:
df.groupBy("Year", "BranchName").agg(sum("Units_Sold").alias("Total_Units")).sort("Year", "Total_Units", ascending=[1,0]).display()

Databricks visualization. Run in Databricks to view.

In [0]:
#df.groupBy("Year", "BranchName").agg(sum("Units_Sold").alias("Total_Units_Sold")).orderBy(col("Year").asc(), col("Total_Units_Sold").desc()).display()

# Write DATA

In [0]:
df.write.format("parquet")\
        .mode("overwrite")\
        .option("path", "abfss://silver@storageadlsgen3.dfs.core.windows.net/carsales")\
        .save()

# Quering Silver Data

In [0]:
df = spark.read.format("parquet").load("abfss://silver@storageadlsgen3.dfs.core.windows.net/carsales")

In [0]:
#df.createOrReplaceTempView("carsales")

In [0]:
%sql
SELECT * FROM parquet.`abfss://silver@storageadlsgen3.dfs.core.windows.net/carsales`